In [7]:
import sys
import ray
import time
import math
import os


In [8]:
ray.init()

2022-07-26 23:32:35,628	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.28.6.160',
 'raylet_ip_address': '172.28.6.160',
 'redis_address': '172.28.6.160:6379',
 'object_store_address': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707',
 'metrics_export_port': 60383,
 'node_id': '1586b34fa8edaba7ad876283501ca54c33f9f528'}

In [ ]:
prefix = ""
@ray.remote
def launch(cc, trace, slow_unit, ack_highprio = 1, abc_dt = 64000, abc_delta = 64000, abc_token = 50):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation"
    os.chdir(work_dir)
    print("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {} --ack_highprio {} --abc_dt {}  --abc_delta {} --abc_token {}".format(cc, trace, slow_unit, ack_highprio, abc_dt, abc_delta, abc_token))
    os.system("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {} --ack_highprio {} --abc_dt {}  --abc_delta {} --abc_token {}".format(cc, trace, slow_unit, ack_highprio, abc_dt, abc_delta, abc_token))


def gen_trafic_config(wkld, nhost, load, time_s, incast, pattern=0):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/traffic_gen"
    os.chdir(work_dir)
    output_name = "{}_n{}_load{}_t{}_incast{}_pattern{}".format(wkld, nhost, load, time_s, incast, pattern)
    cmd = "python traffic_gen.py -c {}.txt -n {} -l {} -b 100G -t {} -o {}.txt -i {} -p {}".format(wkld, nhost, load, time_s, output_name, incast, pattern)
    os.system(cmd)
    mix_path =  work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation/mix"
    os.system("cp ./{}.txt {}".format(output_name, mix_path))
    return output_name

In [92]:
prefix = "1longflow"
lst = []
for abc_delta in [16000, 32000, 64000]:
    f_id = launch.remote("abc", prefix, slow_unit= 0, ack_highprio = 1, abc_dt = abc_delta, abc_delta = abc_delta, abc_token=50)
    lst.append(f_id)
    f_id = launch.remote("abc", prefix, slow_unit= 0, ack_highprio = 1, abc_dt = int(abc_delta / 2), abc_delta = abc_delta, abc_token=50)
    lst.append(f_id)
    f_id = launch.remote("abc", prefix, slow_unit= 0, ack_highprio = 1, abc_dt = 0, abc_delta = abc_delta, abc_token=50)
    lst.append(f_id)
    ray.wait(lst, num_returns=len(lst))  
    lst = []

(pid=38326) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 8000  --abc_delta 16000 --abc_token 50
(pid=38354) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 50
(pid=38401) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 16000  --abc_delta 16000 --abc_token 50


(pid=38326) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) 'build' finished successfully (0.380s)
(pid=38401) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) 'build' finished successfully (0.382s)
(pid=38354) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) 'build' finished successfully (0.386s)


(pid=38326) ENABLE_QCN			Yes
(pid=38326) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38326) PACKET_PAYLOAD_SIZE		1000
(pid=38326) TOPOLOGY_FILE			mix/2layer.txt
(pid=38326) FLOW_FILE			mix/1longflow.txt
(pid=38326) TRACE_FILE			mix/trace.txt
(pid=38326) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt8000dl16000token50_1.tr
(pid=38326) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt8000dl16000token50_1.txt
(pid=38326) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt8000dl16000token50_1.txt
(pid=38326) SIMULATOR_STOP_TIME		2.1
(pid=38326) CC_MODE		9
(pid=38326) ALPHA_RESUME_INTERVAL		1
(pid=38326) RATE_DECREASE_INTERVAL		4
(pid=38326) CLAMP_TARGET_RATE		No
(pid=38326) RP_TIMER			300
(pid=38326) EWMA_GAIN			0.00390625
(pid=38326) FAST_RECOVERY_TIMES		1
(pid=38326) RATE_AI				20Mb/s
(pid=38326) RATE_HAI			200Mb/s
(pid=38326) MIN_RATE		1000Mb/s
(pid=38326) DCTCP_RATE_AI				1000Mb/s
(pid=38326) ERROR_RATE_PER_LINK		0
(pid=38326) L2_CHUNK_SIZE			4000
(pid=38326) L2_ACK_INTERVAL			1
(pid=38326) L2_B

(pid=38326) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) 'build' finished successfully (0.368s)
(pid=38401) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) 'build' finished successfully (0.360s)
(pid=38354) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) 'build' finished successfully (0.357s)


(pid=38326) ENABLE_QCN			Yes
(pid=38326) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38326) PACKET_PAYLOAD_SIZE		1000
(pid=38326) TOPOLOGY_FILE			mix/2layer.txt
(pid=38326) FLOW_FILE			mix/1longflow.txt
(pid=38326) TRACE_FILE			mix/trace.txt
(pid=38326) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt0dl32000token50_1.tr
(pid=38326) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt0dl32000token50_1.txt
(pid=38326) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt0dl32000token50_1.txt
(pid=38326) SIMULATOR_STOP_TIME		2.1
(pid=38326) CC_MODE		9
(pid=38326) ALPHA_RESUME_INTERVAL		1
(pid=38326) RATE_DECREASE_INTERVAL		4
(pid=38326) CLAMP_TARGET_RATE		No
(pid=38326) RP_TIMER			300
(pid=38326) EWMA_GAIN			0.00390625
(pid=38326) FAST_RECOVERY_TIMES		1
(pid=38326) RATE_AI				20Mb/s
(pid=38326) RATE_HAI			200Mb/s
(pid=38326) MIN_RATE		1000Mb/s
(pid=38326) DCTCP_RATE_AI				1000Mb/s
(pid=38326) ERROR_RATE_PER_LINK		0
(pid=38326) L2_CHUNK_SIZE			4000
(pid=38326) L2_ACK_INTERVAL			1
(pid=38326) L2_BACK_TO_ZE

(pid=38326) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) 'build' finished successfully (0.361s)
(pid=38401) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) 'build' finished successfully (0.363s)
(pid=38354) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) 'build' finished successfully (0.364s)


(pid=38326) ENABLE_QCN			Yes
(pid=38326) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38326) PACKET_PAYLOAD_SIZE		1000
(pid=38326) TOPOLOGY_FILE			mix/2layer.txt
(pid=38326) FLOW_FILE			mix/1longflow.txt
(pid=38326) TRACE_FILE			mix/trace.txt
(pid=38326) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt0dl64000token50_1.tr
(pid=38326) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt0dl64000token50_1.txt
(pid=38326) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt0dl64000token50_1.txt
(pid=38326) SIMULATOR_STOP_TIME		2.1
(pid=38326) CC_MODE		9
(pid=38326) ALPHA_RESUME_INTERVAL		1
(pid=38326) RATE_DECREASE_INTERVAL		4
(pid=38326) CLAMP_TARGET_RATE		No
(pid=38326) RP_TIMER			300
(pid=38326) EWMA_GAIN			0.00390625
(pid=38326) FAST_RECOVERY_TIMES		1
(pid=38326) RATE_AI				20Mb/s
(pid=38326) RATE_HAI			200Mb/s
(pid=38326) MIN_RATE		1000Mb/s
(pid=38326) DCTCP_RATE_AI				1000Mb/s
(pid=38326) ERROR_RATE_PER_LINK		0
(pid=38326) L2_CHUNK_SIZE			4000
(pid=38326) L2_ACK_INTERVAL			1
(pid=38326) L2_BACK_TO_ZE

In [91]:
 f_id = launch.remote("abc", prefix, slow_unit= 0, ack_highprio = 1, abc_dt = 0, abc_delta = 16000, abc_token=50)

(pid=38401) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 50


(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) [ 445/1425] install-ns3-header: src/point-to-point/model/switch-node.h -> build/ns3/switch-node.h
(pid=38401) [ 689/1425] cxx: scratch/third.cc -> build/scratch/third.cc.3.o
(pid=38401) [ 788/1425] cxx: src/point-to-point/model/switch-node.cc -> build/src/point-to-point/model/switch-node.cc.1.o
(pid=38401) [1395/1425] cxxshlib: build/src/point-to-point/model/point-to-point-net-device.cc.1.o build/src/point-to-point/model/point-to-point-channel.cc.1.o build/src/point-to-point/model/point-to-point-remote-channel.cc.1.o build/src/point-to-point/model/ppp-header.cc.1.o build/src/point-to-point/helper/point-to-point-helper.cc.1.o build/src/point-to-point/helper/qbb-helper.cc.1.o build/src/point-to-point/model/qbb-net-device.cc.1.o build/src/point-to-point/model/pause-header.cc.1.o build/src/point-to-point/model/cn-header.cc.1.o build/src/point-to-point/model/qbb-header.cc.1.o b

(pid=38401) [1406/1425] cxxshlib: build/src/olsr/model/olsr-header.cc.1.o build/src/olsr/model/olsr-state.cc.1.o build/src/olsr/model/olsr-routing-protocol.cc.1.o build/src/olsr/helper/olsr-helper.cc.1.o -> build/libns3.18-olsr-debug.so
(pid=38401) [1407/1425] cxxshlib: build/src/lte/model/lte-common.cc.1.o build/src/lte/model/lte-spectrum-phy.cc.1.o build/src/lte/model/lte-spectrum-signal-parameters.cc.1.o build/src/lte/model/lte-phy.cc.1.o build/src/lte/model/lte-enb-phy.cc.1.o build/src/lte/model/lte-ue-phy.cc.1.o build/src/lte/model/lte-spectrum-value-helper.cc.1.o build/src/lte/model/lte-amc.cc.1.o build/src/lte/model/lte-enb-rrc.cc.1.o build/src/lte/model/lte-ue-rrc.cc.1.o build/src/lte/model/lte-rlc-sap.cc.1.o build/src/lte/model/lte-rlc.cc.1.o build/src/lte/model/lte-rlc-sequence-number.cc.1.o build/src/lte/model/lte-rlc-header.cc.1.o build/src/lte/model/lte-rlc-am-header.cc.1.o build/src/lte/model/lte-rlc-um.cc.1.o build/src/lte/model/lte-rlc-am.cc.1.o build/src/lte/model/lte-

(pid=38401) ENABLE_QCN			Yes
(pid=38401) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38401) PACKET_PAYLOAD_SIZE		1000
(pid=38401) TOPOLOGY_FILE			mix/2layer.txt
(pid=38401) FLOW_FILE			mix/1longflow.txt
(pid=38401) TRACE_FILE			mix/trace.txt
(pid=38401) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt0dl16000token50_1.tr
(pid=38401) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt0dl16000token50_1.txt
(pid=38401) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt0dl16000token50_1.txt
(pid=38401) SIMULATOR_STOP_TIME		3
(pid=38401) CC_MODE		9
(pid=38401) ALPHA_RESUME_INTERVAL		1
(pid=38401) RATE_DECREASE_INTERVAL		4
(pid=38401) CLAMP_TARGET_RATE		No
(pid=38401) RP_TIMER			300
(pid=38401) EWMA_GAIN			0.00390625
(pid=38401) FAST_RECOVERY_TIMES		1
(pid=38401) RATE_AI				20Mb/s
(pid=38401) RATE_HAI			200Mb/s
(pid=38401) MIN_RATE		1000Mb/s
(pid=38401) DCTCP_RATE_AI				1000Mb/s
(pid=38401) ERROR_RATE_PER_LINK		0
(pid=38401) L2_CHUNK_SIZE			4000
(pid=38401) L2_ACK_INTERVAL			1
(pid=38401) L2_BACK_TO_ZERO

In [70]:
prefix = "1longflow"
abc_dt = 32000
abc_delta = 32000
ack = 1
lst = []
for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
    trace = prefix
    if (method != "slow"):   
        f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
        lst.append(f_id)
    else:
        f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
        lst.append(f_id)
ray.wait(lst, num_returns=len(lst))


(pid=38421) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 1 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38408) python run.py --c dctcp --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38377) python run.py --c hp --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38373) python run.py --c timely --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38415) python run.py --c dcqcn --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38404) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000


(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38408) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38373) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38415) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) 'build' finished successfully (0.385s)
(pid=38377) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) 'build' finished successfully (0.382s)
(pid=38408) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-

(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/1longflow.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abc_slowUnitdt32000dl32000_1.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abc_slowUnitdt32000dl32000_1.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abc_slowUnitdt32000dl32000_1.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		9
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.00390625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				20Mb/s
(pid=38421) RATE_HAI			200Mb/s
(pid=38421) MIN_RATE		1000Mb/s
(pid=38421) DCTCP_RATE_AI				1000Mb/s
(pid=38421) ERROR_RATE_PER_LINK		0
(pid=38421) L2_CHUNK_SIZE			4000
(pid=38421) L2_ACK_INTERVAL			1
(pid=38

(pid=38421) maxRtt=8320 maxBdp=104000
(pid=38421) Running Simulation.
(pid=38408) maxRtt=8320 maxBdp=104000
(pid=38408) Running Simulation.
(pid=38377) maxRtt=8320 maxBdp=104000
(pid=38377) Running Simulation.
(pid=38373) maxRtt=8320 maxBdp=104000
(pid=38373) Running Simulation.
(pid=38415) maxRtt=8320 maxBdp=104000
(pid=38415) Running Simulation.
(pid=38404) maxRtt=8320 maxBdp=104000
(pid=38404) Running Simulation.
(pid=38408) 226.791
(pid=38373) 240.714
(pid=38377) 243.701
(pid=38404) 244.125
(pid=38421) 249.58
(pid=38415) 265.745
without incast: 70443
with incast: 70443
(pid=38421) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38408) python run.py --c dcqcn --trace WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000
(pid=38377) python run

(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38408) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38415) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38373) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) 'build' finished successfully (0.362s)
(pid=38377) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) 'build' finished successfully (0.365s)
(pid=38408) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-

(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0_abcdt32000dl32000_1.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0_abcdt32000dl32000_1.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_WebSearch_distribution_n64_load0.3_t0.5_incast0_pattern0_abcdt32000dl32000_1.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		9
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.00390625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				20Mb/s
(pid=38421) RATE_HAI			200Mb/s
(pid=38421) MIN

(pid=38377) maxRtt=8320 maxBdp=104000
(pid=38377) Running Simulation.
(pid=38408) maxRtt=8320 maxBdp=104000
(pid=38408) Running Simulation.
(pid=38415) maxRtt=8320 maxBdp=104000
(pid=38415) Running Simulation.
(pid=38421) maxRtt=8320 maxBdp=104000
(pid=38421) Running Simulation.
(pid=38373) maxRtt=8320 maxBdp=104000
(pid=38373) Running Simulation.
(pid=38404) maxRtt=8320 maxBdp=104000
(pid=38404) Running Simulation.


KeyboardInterrupt: 

In [94]:
lst = []
for wkld in ["WebSearch_distribution", "FbHdp_distribution"]:
    for load in [0.3, 0.6]:
        for p in [0]:
            for abc_token in [30, 50, 70]:
                prefix = gen_trafic_config(wkld=wkld, nhost=64, load=load, time_s=0.05, incast=0, pattern=p)
                for method in ["abc", "slow"]:
                    trace = prefix
                    if (method != "slow"):   
                        f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = 0, abc_delta = 16000, abc_token = abc_token)
                        lst.append(f_id)
                    else:
                        f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = 0, abc_delta = 16000, abc_token = abc_token)
                        lst.append(f_id)
            ray.wait(lst, num_returns=len(lst))
            lst = []
            
# for wkld in ["WebSearch_distribution", "FbHdp_distribution"]:
#     for load in [0.3]:
#         for p in [0]:
#             prefix = gen_trafic_config(wkld=wkld, nhost=64, load=load, time_s=0.05, incast=1, pattern=p)
#             lst = []
#             for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
#                 trace = prefix
#                 if (method != "slow"):   
#                     f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
#                     lst.append(f_id)
#                 else:
#                     f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
#                     lst.append(f_id)
#             #ray.wait(lst, num_returns=len(lst))


without incast: 6920
with incast: 6920
(pid=38326) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 1 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 30
(pid=38354) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 30
without incast: 6874
with incast: 6874
(pid=38421) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 1 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 50
(pid=38401) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 50


(pid=38354) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'


without incast: 6879
with incast: 6879
(pid=38377) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 70
(pid=38412) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 1 --ack_highprio 1 --abc_dt 0  --abc_delta 16000 --abc_token 70


(pid=38377) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38412) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) 'build' finished successfully (0.388s)
(pid=38354) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) 'build' finished successfully (0.388s)


(pid=38326) ENABLE_QCN			Yes
(pid=38326) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38326) PACKET_PAYLOAD_SIZE		1000
(pid=38326) TOPOLOGY_FILE			mix/2layer.txt
(pid=38326) FLOW_FILE			mix/WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0.txt
(pid=38326) TRACE_FILE			mix/trace.txt
(pid=38326) TRACE_OUTPUT_FILE		mix/mix_2layer_WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token30_1.tr
(pid=38326) FCT_OUTPUT_FILE		mix/fct_2layer_WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token30_1.txt
(pid=38326) PFC_OUTPUT_FILE				mix/pfc_2layer_WebSearch_distribution_n64_load0.3_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token30_1.txt
(pid=38326) SIMULATOR_STOP_TIME		2.1
(pid=38326) CC_MODE		9
(pid=38326) ALPHA_RESUME_INTERVAL		1
(pid=38326) RATE_DECREASE_INTERVAL		4
(pid=38326) CLAMP_TARGET_RATE		No
(pid=38326) RP_TIMER			300
(pid=38326) EWMA_GAIN			0.00390625
(pid=38326) FAST_RECOVERY_TIMES		1
(pid=38326) RATE_AI				20Mb/s
(pid=3

...

(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token30_1.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token30_1.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token30_1.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		9
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.00390625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				20Mb/s
(pid=38421) RATE_HAI			200Mb/s
(pid=38421) MIN_RA

(pid=38401) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38401) 'build' finished successfully (0.384s)
(pid=38354) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38354) 'build' finished successfully (0.381s)


(pid=38354) ENABLE_QCN			Yes
(pid=38354) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38354) PACKET_PAYLOAD_SIZE		1000
(pid=38354) TOPOLOGY_FILE			mix/2layer.txt
(pid=38354) FLOW_FILE			mix/FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0.txt
(pid=38354) TRACE_FILE			mix/trace.txt
(pid=38354) TRACE_OUTPUT_FILE		mix/mix_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token50_1.tr
(pid=38354) FCT_OUTPUT_FILE		mix/fct_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token50_1.txt
(pid=38354) PFC_OUTPUT_FILE				mix/pfc_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abc_slowUnitdt0dl16000token50_1.txt
(pid=38354) SIMULATOR_STOP_TIME		2.1
(pid=38354) CC_MODE		9
(pid=38354) ALPHA_RESUME_INTERVAL		1
(pid=38354) RATE_DECREASE_INTERVAL		4
(pid=38354) CLAMP_TARGET_RATE		No
(pid=38354) RP_TIMER			300
(pid=38354) EWMA_GAIN			0.00390625
(pid=38354) FAST_RECOVERY_TIMES		1
(pid=38354) RATE_AI				20Mb/s
(pid=38354) RATE_HAI		

(pid=38377) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38412) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38377) 'build' finished successfully (0.378s)
(pid=38412) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38412) 'build' finished successfully (0.377s)


(pid=38377) ENABLE_QCN			Yes
(pid=38377) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38377) PACKET_PAYLOAD_SIZE		1000
(pid=38377) TOPOLOGY_FILE			mix/2layer.txt
(pid=38377) FLOW_FILE			mix/FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0.txt
(pid=38377) TRACE_FILE			mix/trace.txt
(pid=38377) TRACE_OUTPUT_FILE		mix/mix_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token70_1.tr
(pid=38377) FCT_OUTPUT_FILE		mix/fct_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token70_1.txt
(pid=38377) PFC_OUTPUT_FILE				mix/pfc_2layer_FbHdp_distribution_n64_load0.6_t0.05_incast0_pattern0_abcdt0dl16000token70_1.txt
(pid=38377) SIMULATOR_STOP_TIME		2.1
(pid=38377) CC_MODE		9
(pid=38377) ALPHA_RESUME_INTERVAL		1
(pid=38377) RATE_DECREASE_INTERVAL		4
(pid=38377) CLAMP_TARGET_RATE		No
(pid=38377) RP_TIMER			300
(pid=38377) EWMA_GAIN			0.00390625
(pid=38377) FAST_RECOVERY_TIMES		1
(pid=38377) RATE_AI				20Mb/s
(pid=38377) RATE_HAI			200Mb/s
(pid=38377) MIN_RA

In [87]:
f_id = launch.remote("abc", "1longflow", slow_unit=0, ack_highprio = 1)
                       

(pid=38326) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 64000  --abc_delta 64000 --abc_token 50


(pid=38326) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38326) 'build' finished successfully (0.380s)


(pid=38326) ENABLE_QCN			Yes
(pid=38326) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38326) PACKET_PAYLOAD_SIZE		1000
(pid=38326) TOPOLOGY_FILE			mix/2layer.txt
(pid=38326) FLOW_FILE			mix/1longflow.txt
(pid=38326) TRACE_FILE			mix/trace.txt
(pid=38326) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt64000dl64000token50_1.tr
(pid=38326) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt64000dl64000token50_1.txt
(pid=38326) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt64000dl64000token50_1.txt
(pid=38326) SIMULATOR_STOP_TIME		3
(pid=38326) CC_MODE		9
(pid=38326) ALPHA_RESUME_INTERVAL		1
(pid=38326) RATE_DECREASE_INTERVAL		4
(pid=38326) CLAMP_TARGET_RATE		No
(pid=38326) RP_TIMER			300
(pid=38326) EWMA_GAIN			0.00390625
(pid=38326) FAST_RECOVERY_TIMES		1
(pid=38326) RATE_AI				20Mb/s
(pid=38326) RATE_HAI			200Mb/s
(pid=38326) MIN_RATE		1000Mb/s
(pid=38326) DCTCP_RATE_AI				1000Mb/s
(pid=38326) ERROR_RATE_PER_LINK		0
(pid=38326) L2_CHUNK_SIZE			4000
(pid=38326) L2_ACK_INTERVAL			1
(pid=38326) L2_

In [ ]:
lst = []
for wkld in ["FbHdp_distribution"]:
    for load in [0.6]:
        for p in [0]:
            for token in [30, 40, 50, 60]:
                for abc_delta in [16000, 32000, 64000]
                    prefix = gen_trafic_config(wkld=wkld, nhost=64, load=load, time_s=0.05, incast=0, pattern=p)
                    for method in ["abc"]:
                        trace = prefix
                        f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_delta, abc_delta = abc_delta, abc_token = abc_token)
                        lst.append(f_id)
                        f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = 0, abc_delta = abc_delta, abc_token = abc_token)
                        lst.append(f_id)    
                ray.wait(lst, num_returns=len(lst))
                lst = []